<a href="https://colab.research.google.com/github/tmoura/softexIA/blob/main/MLflow-6-Registry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mlflow --quiet
!pip install pyngrok --quiet

In [ ]:
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

import pandas as pd

url = "https://raw.githubusercontent.com/tmoura/machinelearning/master/datasets/iris.data"

# Carregar base de dados
# DataFrame
dataset = pd.read_csv(url, header=None)

columns = len(dataset.columns)

y = dataset[0] # extrai a primeira coluna, que é o label
X = dataset.loc[:,1:columns-1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None, stratify=y) # 80% treino e 20% teste

# *Model Sklearn*

In [ ]:
import mlflow

mlflow.end_run()

exp1 = mlflow.set_experiment(experiment_name="experimento_registrymodel")

mlflow.start_run(run_name="Run1.1",experiment_id=exp1.experiment_id)
model = tree.DecisionTreeClassifier()
model = model.fit(X_train, y_train)

result = model.predict(X_test)
acc = metrics.accuracy_score(result, y_test)

mlflow.log_param("Criterion", "entropy")
mlflow.log_param("MAX_DEPTH", 2)
mlflow.log_metric("Acurácia", acc)
mlflow.sklearn.log_model(model, "treemodelENTROPY")

mlflow.register_model(
      model_uri='runs:/{}/model'.format(run.info.run_id),
      name='elastic-api-2'
)

##### Informações sobre o experimento

print("Nome: {}".format(exp1.name))
print("ID do experimento: {}".format(exp1.experiment_id))
print("Local dos Artefatos: {}".format(exp1.artifact_location))
print("Tags: {}".format(exp1.tags))
print("Estágio do Ciclo de Vida: {}".format(exp1.lifecycle_stage))
print("Tempo de criação: {}".format(exp1.creation_time))

mlflow.end_run()

get_ipython().system_raw("mlflow ui --port 5000 &") # run tracking UI in the background

# *Load Model SkLearn*

In [ ]:
# LOAD MODEL SkLEARN

modelNEW = mlflow.sklearn.load_model(model_uri="models:/elastic-api-2/1")

result = modelNEW.predict(X_test)
acc = metrics.accuracy_score(result, y_test)

show = round(acc * 100)

print("Acurácia de: {}%".format(show))

# Uso do NGROK

O Ngrok é uma ferramenta que permite criar um túnel seguro para conectar um servidor local a um servidor remoto, tornando possível expor uma aplicação local para a Internet. Essa ferramenta é muito útil para desenvolvedores que precisam testar suas aplicações localmente antes de colocá-las em produção.

In [ ]:
from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2fbGojiXWPL7flUUKztsxbdTdrP_3DXnMDuQVRd5G5n6Fhr6A"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("Endereço do MLflow:", ngrok_tunnel.public_url)

Endereço do MLflow: https://4cca-34-106-112-134.ngrok-free.app
